<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>

***

# 1.0 Introduction to ROS Robot Control

In this section, you will learn some basics about the ROS framework, and how ROS processes, or "nodes", communicate with each other and the robot hardware by publishing and subscribing to "topics". You'll work through some hands-on exercises with a Gazebo simulator of the robot, using the desktop Linux Ubuntu window. Ultimately, you will complete Python code for your own `bot-nav` ROS node, test it in simulation, and deploy it to the physical robot's NVIDIA Jetson. 

[1.1 What is ROS?](#ros)<br>
[1.2 What is Gazebo?](#gazebo)<br>
[1.3 Getting Started](#gettingstarted)<br>
[1.4 Your ROS Control Node: bot-nav mobility](#controlnode)<br>
[1.5 Assessment](#assessment)<br>
[1.6 Deployment](#deployment)<br>
[1.7 Recap](#recap)<br>
[Appendix](9900_DL4R_Appendix.ipynb#apx-1)

<a id='ros'></a>
# 1.1 What is the Robot Operating System (ROS)?
As defined in the [ROS Wiki](http://wiki.ros.org/ROS/Introduction): <br>
>ROS is an open-source, meta-operating system for your robot. It provides the services you would expect from an operating system, including hardware abstraction, low-level device control, implementation of commonly-used functionality, message-passing between processes, and package management. It also provides tools and libraries for obtaining, building, writing, and running code across multiple computers. 

At runtime, a ROS robotic system can be visualized as a graph of processes, or **ROS nodes** connected by message streams, or **ROS topics**.  The ROS system is controlled by a special **ROS Master** node.  The ROS Master provides name registration and lookup to the rest of the ROS nodes in the graph.  The Master provides the mechanism for finding registered nodes, exchanging messages, and invoking services.

<img src="images/ROSnodes1.png" width=600/>

In this example, the `Camera Node` receives raw data from the camera and publishes it to a ROS topic, `/image_data`.  The information can now be subscribed to by any number of ROS nodes.  In this diagram, the Image Processing node has subscribed to `/image_data`.

### Nodes
Each ROS node provides computation for some component of the larger robotic system.  For example, one node may control a camera, another the robot's wheel motor, another the image processing or localizaion, and so on.  The ROS Nodes are self-contained and may be written in different languages, even within the same robot.  A ROS client library is imported into the code, typically for C++ (library is `roscc`) or Python (library is `rospy`). 

### Messages
Nodes communicate with each other by passing messages, which consist of defined data structures of typed fields.  Available primitives include integers, floats, boolean, strings, etc. More options can be found on the [ROS msg](http://wiki.ros.org/msg) page.

### Topics
A node sends a message by _publishing_ it to a topic.  A node receives a message by _subscribing_ to that topic.  Multiple nodes may publish to the same topic and multiple nodes may subscribe to any topic.  The format of the data that is published is strongly typed. Nodes are independent, in that they are unaware of which other nodes might be providing or using a given topic.

<a id='gazebo'></a>
# 1.2 What is Gazebo?
As defined in the [Gazebo](http://gazebosim.org/tutorials?tut=guided_b1&cat=) tutorials:
>Gazebo is a 3D dynamic simulator with the ability to accurately and efficiently simulate populations of robots in complex indoor and outdoor environments. While similar to game engines, Gazebo offers physics simulation at a much higher degree of fidelity, a suite of sensors, and interfaces for both users and programs.

Robot simulation makes it possible to rapidly test new functionality in a dynamic realistic environment.  There are some things, including actual floor friction and the inference itself, that we cannot truly simulate in the environment we have created, but we can at least work through the functionality prior to deployment!  

<img src="images/sim-dev-dep.png"/>

<a id='gettingstarted'></a>
# 1.3 Getting Started

### <a href="/vnc/" target="_blank">OPEN DESKTOP</a>
First, click the link above to launch the desktop.<br>
 
A new tab will a open in your browser with "NoVNC" and a **Connect** button.

<img src="images/noVNC.png" alt="noVNC" width=400/>

Click the **Connect** button and enter the password "nvidia" to log in.
An Ubuntu desktop will appear.  At the bottom of the page are some launch buttons for various utilities you can run inside the  desktop.  Click the terminal icon (second from the left) to open a desktop shell terminal.

<img src="images/desktopApps2.png" alt="desktop-apps" width=400/>

In the instructions that follow, you will be asked to enter lines into terminal windows.  The remote desktop does not accept copy/paste directly from your local computer, but it can be done with a two-step process:<br>

1) On the left side of the remote desktop there is a slider with a clipboard tool that allows you to copy/paste from your local computer to the remote clipboard.
2) Once a line is copied to the remote clipboard, it can be pasted as usual into a terminal window.

<img src="images/copy_paste.png" alt="copy_paste" />

### Launch the Simulator
The environment has been preconfigured for the robot simulation.  ROS, Gazebo, and the Robot Model for the AION ROBOTICS R1 have already been set up.  If you wish to set up your own native environment, information to get started is included in the <a href=#apx-environmet-setup>Appendix</a>. To start the simulator, enter the following command in the terminal window:

<code  style="
  border:2px solid gray;
  background-color:black;
  color:white;
  display: block;
  padding:10px; 
  width:400px; 
  ">roslaunch r1_gazebo gazebo.launch
</code>

ROS nodes can be launched individually or in groups.  The `roslaunch` command starts the ROS master node as well as other ROS nodes.  The parameters are the launch package (`r1_gazebo` in this example) and launch definition file (`gazebo.launch` in this example).  This launch file starts Gazebo with our robot model and robot world.  You will see the simulated robot inside an empty world.  At this point you can zoom in, zoom out, and view the robot from a number of angles using the icons at the top of the screen.  Go ahead and try it!

There is no controller node present at this point to connect the robot to the wheels and steering, so you can't make it go yet. 

<img src="images/aion_gazebo.png" alt="desktop-apps"/>

### List the ROS nodes
To see which nodes and topics have been launched, open a new terminal and enter the following:

<code  style="
  border:2px solid gray;
  background-color:black;
  color:white;
  display: block;
  padding:10px; 
  width:400px; 
  ">rosnode list
</code>

The `rosnode` command with the `list` subcommand lists all the active ROS nodes. These nodes represent the infrastructure for the Gazebo simulation. 

<img src="images/rosnode_list.png" width=600/>

### List the ROS topics
Now let's see a list of the current topics in the system.  Enter the following command in the terminal window:

<code  style="
  border:2px solid gray;
  background-color:black;
  color:white;
  display: block;
  padding:10px; 
  width:400px; 
  ">rostopic list
</code>

<img src="images/rostopic_list.png" alt="desktop-apps" width=600/>


There are a number of topics available.  The one we are most interested in is the `/cmd_vel_mux/input/teleop` topic.  By publishing to this topic, we can control the robot, because the drive controller in both the simulated robot and the physical robot are subscribing to this topic for movement commands.  If we write to it, the robot will move.


<a id='visualize'></a>
### Visualize the ROS graph
Next, let's see a visualization of the **ROS graph**, which shows ROS nodes (ovals) and their connectors, the ROS topics (rectangles). To see this, we will use the `rosrun` command to add a special ROS node , `rqt_graph`, in the `rqt_graph` package, to the ROS network. Enter the following in the terminal:

<code  style="
  border:2px solid gray;
  background-color:black;
  color:white;
  display: block;
  padding:10px; 
  width:400px; 
  ">rosrun rqt_graph rqt_graph
</code>


<img src="images/rosgraph_noteleop.png" alt="desktop-apps" width=600/>

In the upper left, make sure the `Nodes/Topics(all)` is selected for the view.  As you can see, there are no nodes publishing to the `/cmd_vel_mux/input/teleop` topic, so the robot can't move.
<br>
Kill the graph node by entering `CTRL C` in the terminal window that you launched it from.

### Drive the R1 inside the simulator
A keyboard ROS "teleop" controller has already been set up in the workspace.  Add the teleop ROS node from a new terminal window:

<code  style="
  border:2px solid gray;
  background-color:black;
  color:white;
  display: block;
  padding:10px; 
  width:600px; 
  ">rosrun teleop_twist_keyboard teleop_twist_keyboard.py
</code>

Keep your cursor in this terminal window to keep the keyboard controller node active. Move the robot around with the "Moving around" keyboard keys as listed in the terminal window.

<img src="images/teleop_keys.png" alt="teleop_keys" width=600 align="left"/>

Go ahead and experiment a bit.  If you happen to run your robot off the screen, get it repositioned using the upper left corner of Gazebo "Edit"->"Reset Model Poses" pull-down command.

<img src="images/reset_pose.png" alt="reset_pose" width=800/>


As you move the robot around in simulation, the simulated controller sends messages to the simulated robot.  These are the same messages that would be sent in the real world.  

Once you are comfortable with moving the robot around, open a new terminal window and take a look at the graph once again by launching the `rqt_graph` node as before (Do you remember how? Refer back to [Visualize a ROS graph](#visualize) section)

<img src="images/rosgraph_with_teleop.png" alt="desktop-apps" width=600/>

### Look at the Twist messages

Now the new ROS node `/teleop_twist_keyboard` is shown, and it is publishing to the `/cmd_vel_mux/input/teleop` topic.  You can now control the robot with the simple keyboard controller. 

What do these messages look like?  We can look at any topic with the `rostopic echo` command and subcommand. Go ahead and kill the `rqt_graph` node with a `CTRL-C` in that terminal window, and enter the following to see the `/cmd_vel_mux/input/teleop` messages:

<code  style="
  border:2px solid gray;
  background-color:black;
  color:white;
  display: block;
  padding:10px; 
  width:400px; 
  ">rostopic echo /cmd_vel_mux/input/teleop
</code>

Nothing will show until you send a command to the robot from your keyboard.  Select the terminal window that is running the `teleop_twist_keyboard` node and move the robot with the keyboard as before.  Now you should see the message data in the echo window.







<img src="images/teleop_echo.png" alt="desktop-apps"/>

The message data you see is in the format of the ROS predefined [Geometry_msgs/Twist](http://docs.ros.org/melodic/api/geometry_msgs/html/msg/Twist.html) message.  Next, you will write a controller node for automatic control of the robot using this type of message.

<a id='controlnode'></a>
# 1.4 Your ROS Control Node: `bot-nav mobility`

To replace the keyboard control, all we need to do is create some other node that publishes the relevant messages to the `cmd_vel_mux/input/teleop` topic, and write our own logic in that node to command the robot as we wish!  This could be via pre-programmed movements, which is what you'll build in this section, or in response to some subscribed topic from a visual inference node, as you will build in the later sections.

You'll find some scaffolding for the control node(s) set up in the `bot-nav` ROS package.  Take a look at the file structure by executing the cell immediately below. 

In [ ]:
!tree -d ~/AionR1_ws/src

The workspace, named `AionR1_ws`, contains ROS packages in the `src` directory.  Looking at the directory tree, we see that our workspace includes the R1 simulation package, `r1_ugv_sim`, the teleop keyboard package we just used, `teleop_twist_keyboard`, and the package for our project, `bot-nav`.  The `bot-nav` package has subdirectories with control nodes for each section of the project. 

Execute the cell immediately below to list the files for section 1. The `mobility` file is a python file, and is the ROS node you'll work with. 

In [ ]:
!ls ~/AionR1_ws/src/bot-nav/scripts/section1

To get a good look at the code itself, as well as to edit your file later, run the following cell to create a symbolic link from this notebook environment to the ROS workspace.

In [ ]:
!ln -sv ~/AionR1_ws/src/bot-nav/scripts/section1/mobility mobility

Next, click the `File/Open` drop-down command from the navigation on this notebook page. This will open another Jupyter page where you can browse all the notebooks for the course, as well as the symbolic link for `mobility` that you just created.  Double click on the `mobility` link to open the file.  To make it more readable, select Python from the drop-down Language menu. Take a few minutes to familiarize yourself with the code.

### `rospy.Publisher` 

ROS provides a python library, `rospy`, which includes the `Publisher` and `Subscriber` objects.  These `rospy.Subscriber` and `rospy.Publisher` objects are the key to the node's communication with other nodes.  [This ROS wiki: Publishers and Subscribers](http://wiki.ros.org/rospy/Overview/Publishers%20and%20Subscribers) provides a great deal of helpful information on how to set up a publishers and subscribers.  The basic usage of the publisher is summarized by this snippet from the wiki:

<img src="images/publish_example.png" alt="publish_example" width=800/>

On the physical robot, the hardware controller node is subscribed to the `/cmd_vel` topic.  Our `mobility` node publishes to this topic by creating a [rospy.Publisher](http://docs.ros.org/api/rospy/html/rospy.topics.Publisher-class.html
) object in the class constructor (look the `__init__` definition of the `mobility` file):

```python
# Publisher to /cmd_vel topic
self.cmd_vel = rospy.Publisher('/cmd_vel', Twist, queue_size=10)
```

The 'topic name' for our node is '/cmd_vel', and the message type is [geometry_msgs/Twist](http://docs.ros.org/api/geometry_msgs/html/msg/Twist.html). If you follow this link defining the `Twist` message structure, you will find that it consists of two [Vector3](http://docs.ros.org/api/geometry_msgs/html/msg/Vector3.html) objects named `linear` and `angular`, each with an `x`, `y`, and `z` `float64` primitive.  You can also find the message format with the `rosmsg` command on the desktop:

<code  style="
  border:2px solid gray;
  background-color:black;
  color:white;
  display: block;
  padding:10px; 
  width:400px; 
  ">rosmsg show geometry_msgs/Twist
</code>

### Mapping the topic from `/cmd_vel` to `cmd_vel_mux/input/teleop`
For our simulator, we need to publish movement commands to the `/cmd_vel_mux/input/teleop` topic, as you've already seen when you used the keyboard teleop.  We'll develop a control node that publishes to the `/cmd_vel` topic and simply map it to `/cmd_vel_mux/input/teleop` topic for simulation purposes.  It turns out this is very easy to do from the `rosrun` command line when we launch the node.  Follow this pattern:  <br>

```bash
rosrun <my_ros_package> <my_ros_node> /cmd_vel:=/cmd_vel_mux/input/teleop
```

It is *important* to do this mapping in order for the simulator to receive the movement commands, because the simulator is *only subscribed to the `/cmd_vel_mux/input/teleop` topic*, and not looking at `/cmd_vel` at all.

### `rospy.Subscriber`
Basic usage of a subscriber is shown in the example below from the ROS wiki page.  

<img src="images/rospy_subscriber_example.png" alt="subscribe_example" width=800>

The "listener" creates the `Subscriber` object and specifies the "callback" definition and format.  In the `mobility` node, the subscriber is set up in this line:

```python
# Subscriber to /remote_command topic
self.receive_remote_command \
    = rospy.Subscriber('/remote_command', String, self.remote_command_callback, queue_size=1)
```

In this case, the node is "listening" to the `/remote_command` topic.  On the physical robot, this topic is initiated by the hand remote controller.  The data type sent is a simple `String` type.  The node recognizes "line", left_turn", "circle", "chicane", "guided", and "not_guided".

When data is received across the `/remote_command` topic, the `self.remote_command_callback` method is automatically invoked.


### Launch the starter `mobility` node and bot-nav interface tool

At this time, go ahead and kill the keyboard teleop node from the previous exercise with a `CTRL-C` and launch the following - you'll need two terminal windows:
* the `mobility` control node with `/cmd_vel` properly mapped for simulation
    

</code><code  style="
  border:2px solid gray;
  background-color:black;
  color:white;
  display: block;
  padding:10px; 
  width:600px; 
  ">rosrun bot-nav mobility /cmd_vel:=/cmd_vel_mux/input/teleop
</code>
    
* the `bot-nav interface` tool

<code  style="
  border:2px solid gray;
  background-color:black;
  color:white;
  display: block;
  padding:10px; 
  width:200px; 
  ">bni
</code>



### The bot-nav Interface tool (`bni`)

The bot-nav interface tool publishes the commands we want to test on a topic named `/remote_command`: "line", "circle", "left turn", and "chicane".  The `mobility` node is subscribed to the `/remote_command` topic and responds.  To see how this works, first check the "Guided Mode" box.  The robot will only run computer guided commands in this mode.  Otherwise it will still look for the human remote control.

* Click the "line" command.  The robot should move forward for a few seconds and then stop.<br>
* Try the "circle" command.  The robot should drive a partial circle to the right.<br>
* Try the "left turn" command.  The robot doesn't move.  That's because there is no code for the left turn yet - that's what you'll work on next!

<img src="images/bot-nav-interface.png" alt="bot-nav_interface" width=400/>

<a id='assessment'></a>
# 1.5 Assessment

### Section 1 Coding: Make your robot respond correctly to the "left turn" command.
Follow the "TODO" instructions in the `~/AionR1_ws/src/bot-nav/scripts/section1/mobility` python ROS node file.

```python
...
    @staticmethod
    def create_left_turn_message(angle_degrees, linear_speed, turn_time):
        move_cmd = Twist()
        # TODO complete the code to build the Twist message for a left turn
        # ********your code starts here - do not modify code above here

        # ********your code ends here - do not modify code below here
        return move_cmd
...
```

Note that any time you edit a Python ROS node, such as `mobility`, you will need to do the following to see your changes take effect:
* Stop the running node with CTRL-C in its terminal
* Relaunch the node exactly as before (remember to map the topic!)
* Unclick and Click the "guided" mode on the `bni` tool

### Monitor the commands
For the purposes of debugging, you can monitor the commands on the `/remote_command` topic.  You've done this before with another topic - here's the command:

<code  style="
  border:2px solid gray;
  background-color:black;
  color:white;
  display: block;
  padding:10px; 
  width:400px; 
  ">rostopic echo /remote_command
</code>

If you want to see how this all looks in a ROS graph, you can open another window and run the `rqt_graph` as before:

<code  style="
  border:2px solid gray;
  background-color:black;
  color:white;
  display: block;
  padding:10px; 
  width:400px; 
  ">rosrun rqt_graph rqt_graph
</code>

<img src="images/rqt_mobility_sim_monitored.png" width=800/>

Once your code is working correctly in the simulator, you can submit it for course assessment.  
First, copy your `mobility` node to the assessment folder by executing the following copy command. You'll need to do this any time you change your code and want to resumbit for grading.   You can do this as many times as you wish.

In [ ]:
!cp -v ~/AionR1_ws/src/bot-nav/scripts/section1/mobility assessment_export_dir

Next, return to the classroom launch page, and click the the checkmark titled "ASSESS TASK" to run the grading assessment.  

<img src="images/assess_task.png" width=800/>

<a id='deployment'></a>
# 1.6. Deployment

You will deploy with a small group of other students.  Choose one of the working `mobility` nodes from among the group and follow class instructions on how to submit the code for deployment.  When it is your group's turn, you will work with a TA to transfer the `mobility` file to the same location on the Jetson TX2, residing on the robot.

On the physical robot, your code will receive `/remote_command` messages from the radio controller device instead of the simulated "bot-nav interface". Your published Twist messages to the `/cmd_vel` topic will control the robot.

During deployment, team member jobs may include:
* Code submission (one file per group)
* Video-taping and sharing the robot in action
* Working the radio remote-control to send commands to the robot
    * <a style="color:red;">IMPORTANT: Don't let the robot run amok!!</a>

<a id='recap'></a>
# 1.7 Recap

Congratulations!  You have successfully written a control node to move the robot without a manual controller.  Along the way you have learned to:
* Launch and run ROS nodes
* Move a robot inside a Simulator
* Create a new ROS control node for a robot
* Analyze the current state of the system with `rosnode` and `rostopic`
* Visualize the ROS graph with `rqt_graph`
* Deploy a ROS node from simulation to a physical robot

Up next, you'll add deep learning capabilities to the robot, so that it can recognize an object and can respond to its environment.  

## [--> Next: 2.0 ROS Integration of Image Recognition](0200_DLforRoboticsImageRecognition.ipynb)

<div style="border:2px solid black; background-color:#f2f2f2; padding:10px; width:400px; margin:auto;">
<b><a href="DLI Welcome Notebook.ipynb">Overview</a></b><br>
<b><a href="0100_DLforRoboticsControl.ipynb">1.0 Introduction to ROS Control</a></b><br>
<b><a href="0200_DLforRoboticsImageRecognition.ipynb">2.0 ROS Integration of Image Recognition</a></b><br>
<b><a href="0300_DLforRoboticsObjectDetection.ipynb">3.0 ROS Integration of Object Detection</a></b><br>
<b><a href="9900_DL4R_Appendix.ipynb">Appendix</a></b><br>
</div>

<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>